In [2]:
def read_file(file_path):
    with open(file_path) as f:
        str_text = f.read()
    return str_text

In [5]:
import spacy
nlp = spacy.load('en_core_web_sm',disable=['parser','tagger','ner'])

nlp.max_length = 1198623

In [7]:
def seperate_punc(doc_text):
    return[token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [10]:
d = read_file('moby_dick_four_chapters.txt')
tokens = seperate_punc(d)

C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [12]:
len(tokens)

11338

In [13]:
tokens[1:5]

['me', 'ishmael', 'some', 'years']

In [15]:
train_len = 25 + 1

text_sequences = []

for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

In [16]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [20]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [21]:
sequences[0]

[956,
 14,
 263,
 51,
 261,
 408,
 87,
 219,
 129,
 111,
 954,
 260,
 50,
 43,
 38,
 314,
 7,
 23,
 546,
 3,
 150,
 259,
 6,
 2713,
 14,
 24]

In [23]:
vocabulary_size = len(tokenizer.word_counts)

In [24]:
vocabulary_size

2718

In [25]:
import numpy as np
sequences = np.array(sequences)

In [26]:
sequences

array([[ 956,   14,  263, ..., 2713,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2712, ...,   53,    2, 2718],
       [ 166, 2712,    3, ...,    2, 2718,   26]])

In [27]:
import tensorflow.keras

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,LSTM

In [29]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size,25,input_length=seq_len))
    model.add(LSTM(150,return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150,activation='relu'))
    
    model.add(Dense(vocabulary_size,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [30]:
from tensorflow.keras.utils import to_categorical

In [31]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [32]:
y = to_categorical(y,num_classes=vocabulary_size+1)

In [33]:
seq_len = X.shape[1]

In [34]:
seq_len

25

In [35]:
model = create_model(vocabulary_size+1, seq_len)

C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [36]:
model.fit(X, y, batch_size=128, epochs=255,verbose=1)

Epoch 1/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 13s 89ms/step - accuracy: 0.0458 - loss: 7.1893
Epoch 2/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 8s 92ms/step - accuracy: 0.0514 - loss: 6.3642
Epoch 3/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 9s 96ms/step - accuracy: 0.0536 - loss: 6.3006
Epoch 4/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 10s 106ms/step - accuracy: 0.0548 - loss: 6.1622
Epoch 5/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 9s 106ms/step - accuracy: 0.0583 - loss: 6.0572
Epoch 6/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 9s 105ms/step - accuracy: 0.0671 - loss: 5.8883
Epoch 7/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 9s 106ms/step - accuracy: 0.0717 - loss: 5.7926
Epoch 8/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 10s 112ms/step - accuracy: 0.0707 - loss: 5.7115
Epoch 9/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 9s 105ms/step - accuracy: 0.0698 - loss: 5.6342
Epoch 10/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 10s 107ms/step - accuracy: 0.0766 - loss: 5.5521
Epoch 11/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 9s 105ms/step - accuracy: 0.0802 - loss: 5.4986
Epoch 12/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 

89/89 ━━━━━━━━━━━━━━━━━━━━ 12s 135ms/step - accuracy: 0.4852 - loss: 2.0168
Epoch 134/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 12s 130ms/step - accuracy: 0.4733 - loss: 2.0409
Epoch 135/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 12s 136ms/step - accuracy: 0.4850 - loss: 2.0014
Epoch 136/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 12s 131ms/step - accuracy: 0.4920 - loss: 1.9814
Epoch 137/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 14s 153ms/step - accuracy: 0.4939 - loss: 1.9680
Epoch 138/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 12s 130ms/step - accuracy: 0.5019 - loss: 1.9356
Epoch 139/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 12s 129ms/step - accuracy: 0.4992 - loss: 1.9258
Epoch 140/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 11s 127ms/step - accuracy: 0.5054 - loss: 1.9180
Epoch 141/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 11s 122ms/step - accuracy: 0.5101 - loss: 1.9004
Epoch 142/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 11s 128ms/step - accuracy: 0.5148 - loss: 1.8666
Epoch 143/255
89/89 ━━━━━━━━━━━━━━━━━━━━ 11s 126ms/step - accuracy: 0.5131 - loss: 1.8723
Epoch 144/255
89/89 ━━━━

In [37]:
from pickle import dump,load

In [38]:
from random import randint
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import pad_sequences

In [47]:
def generate_text(model,tokenizer,seq_len,seed_text,num_gen_words):
    
    output_text = []
    
    input_text = seed_text
    
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        pad_encoded = pad_sequences([encoded_text],maxlen=seq_len,truncating='pre')
        
        pred_word_ind = model.predict(pad_encoded, verbose=0)[0].argmax()
        
        pred_word = tokenizer.index_word[pred_word_ind]
        
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
    
    return ' '.join(output_text)

In [48]:
import random
random.seed(101)
random_pick = randint(0,len(text_sequences))

In [49]:
random_seed_text = text_sequences[random_pick]

In [50]:
seed_text = ' '.join(random_seed_text)

In [51]:
seed_text

"thought i to myself the man 's a human being just as i am he has just as much reason to fear me as i have"

In [52]:
generate_text(model,tokenizer,seq_len,seed_text,25)

'not not turning flukes it but look here cunningly meantime the congealed death principle to overlapping against such doubtless a matter a landlord was near'